# Libs & Util

In [ ]:
# Extracting Features
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
import noisereduce as nr
import IPython
from tqdm.notebook import tqdm
import json

# Training neural networks
import sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Live detection
import pyaudio
from IPython.display import clear_output
import wave

# ms per chunk
STEP = 50
PATH = "whistle_dataset/"

# Extracting features

## Chunk processor

>The chunk processor will get a sample with a feature type, and will utilize these to process said chunk into features. The chunk processor is used in both featurizers.

In [ ]:
#def chunk_processor(sample, sample_rate, feature_type, target, false_window = 1, true_window= 1/50):
#def chunk_processor(sample, sample_rate, feature_type, target, false_window = 10, true_window= 1/5):
def chunk_processor(sample, sample_rate, feature_type, target, false_window = 25, true_window= 1/2):
    
    # get correct window variable
    if target:
        window = true_window
    else:
        window = false_window
        
    # calculate chunk size
    chunk = int((sample_rate / 1000) * STEP)
    
    # iterate over sample and fetch features
    for i in tqdm(range(0, len(sample) - chunk, int(chunk * window)), leave=False):
        if feature_type == "fft":
            chunk_features = np.mean(np.abs(librosa.stft(sample[i:i+chunk], n_fft=512, hop_length=256, win_length=512)).T, axis=0)
        elif feature_type == "mfcc":
            chunk_features = np.mean(librosa.feature.mfcc(y=sample[i:i+chunk], sr=sample_rate, n_mfcc=40).T, axis=0)
        
        try:
            features = np.append(features, np.array([chunk_features]), axis=0)
        except:
            features = np.array([chunk_features])
            
    return features, np.full(len(features), int(target))

## Featurizer V1

> Featurizer 1 only uses the whistle, and the segments before and after (same length). This is how a 2:1 label ratio is managed. Sequential samples also have 50% overlap.

In [ ]:
# old windows were 1/5
def build_feature_dataframe_v1(feature_type, denoise=False):
    """Convert all whistles and small fragments before and after said whistles into features"""
    
    # See if csv has been calculated before (saving time)
    try:
        df = pd.read_csv("data_v1_" + feature_type + "_" + str(denoise) + ".csv", index_col=0)
        print("Dataframe succesfully loaded from csv!")
        return df
    except:
        pass
    
    # get the labels
    target = []
    with open(PATH + "whistledb.json") as json_file:
        labels = json.load(json_file)["audioFiles"]
        labels = {entry["path"] : entry["channels"][1]["whistleLabels"] for entry in labels}
    
    # iterate over all audiofiles
    for file_name in tqdm(os.listdir(PATH)):
        # skip json file
        if file_name.split(".")[-1] != "wav":
            continue

        # load file & meta data
        sample, sample_rate = librosa.load(PATH + file_name, sr=None)
        if denoise == True:
            sample = nr.reduce_noise(y=sample,  y_noise=sample[0:5000], sr=sample_rate)

        # for all positive intervals get part before and after aswell and featurize
        for times in tqdm(labels[file_name], leave=False):
            delta_time = times["end"] - times["start"]
            
            label = False
            for i in range(times["start"]-delta_time, times["end"]+delta_time, delta_time):
                if i >= 0 and i + delta_time <= len(sample):
                    features, targets = chunk_processor(sample[i:i+delta_time], sample_rate, feature_type, label, 
                                                        false_window = 1/2, true_window= 1/2)
                    label = not(label)
                    try:
                        out = np.append(out, features, axis=0)
                        target = np.append(target, targets)
                    except:
                        out = features
                        target = targets
                else:
                    print(file_name, i, 'no fit ;(')

    # save them in dataframe
    df = pd.DataFrame(out)
    df=(df-df.min())/(df.max()-df.min())
    df.insert(0, "target", target)
    df.to_csv("data_v1_" + feature_type + "_" + str(denoise) + ".csv")
    return df

## Featurizer V2

> Featurizer 2 utilises two different overlap values. True labels have 1/2 overlap, meaning that sequential samples have 50% overlap. False labels have a overlap value of 25, meaning that after each sample taken 24 are skipped. This insures that a 2:1 label ratio is kept while still taking a more fair representation of the dataset.

In [ ]:
def build_feature_dataframe_v2(feature_type, denoise=False):
    
    # See if csv has been calculated before (saving time)
    try:
        df = pd.read_csv("data_v2_" + feature_type + "_" + str(denoise) + ".csv", index_col=0)
        print("Dataframe succesfully loaded from csv!")
        return df
    except:
        pass
    
    # get the labels
    target = []
    with open(PATH + "whistledb.json") as json_file:
        labels = json.load(json_file)["audioFiles"]
        labels = {entry["path"] : entry["channels"][1]["whistleLabels"] for entry in labels}
    
    # iterate over all audiofiles
    for file_name in tqdm(os.listdir(PATH)):
        # skip json file
        if file_name.split(".")[-1] != "wav":
            continue

        # load file & meta data
        sample, sample_rate = librosa.load(PATH + file_name, sr=None)
        if denoise == True:
            sample = nr.reduce_noise(y=sample,  y_noise=sample[0:5000], sr=sample_rate)
        
        # create time intervals
        times_list = [0]
        for times in labels[file_name]:
            times_list += [times["start"], times["end"]]
        if times_list[-1] < len(sample):
            times_list.append(len(sample))
        
        label = False
        for i in tqdm(range(len(times_list)-1), leave=False):
            features, targets = chunk_processor(sample[times_list[i]:times_list[i+1]], sample_rate, feature_type, label)
            label = not(label)
            try:
                out = np.append(out, features, axis=0)
                target = np.append(target, targets)
            except:
                out = features
                target = targets

    # save them in dataframe
    df = pd.DataFrame(out)
    df=(df-df.min())/(df.max()-df.min())
    df.insert(0, "target", target)
    df.to_csv("data_v2_" + feature_type + "_" + str(denoise) + ".csv")
    return df

In [ ]:
def build_features(version, feature_type, denoise):
    if version == 1:
        return build_feature_dataframe_v1(feature_type, denoise)
    elif version == 2:
        return build_feature_dataframe_v2(feature_type, denoise)
    else:
        print("invalid version")

In [ ]:
mass_data = build_features(1, "mfcc", False)
print("True label ratio:", round((len(mass_data[mass_data["target"] == 1]) / len(mass_data["target"])) * 100, 1),"%")

## Dataset debugging

> The following code allows for any whistle sample to be extracted in order to check if they are labeled correctly.

In [ ]:
# Choose recording
i = -1
fname = os.listdir(PATH)[i]
print(fname)

with open(PATH + "whistledb.json") as json_file:
    labels = json.load(json_file)["audioFiles"]
    labels = {entry["path"] : entry["channels"][1]["whistleLabels"] for entry in labels}
print(labels[fname], len(labels[fname]))

sample_tts, sr = librosa.load(PATH + fname, sr=None)

In [ ]:
# Choose whistle number
i = -1
print(os.listdir(PATH))
print(labels[fname][i]["start"],labels[fname][i]["end"])

IPython.display.Audio(sample_tts[labels[fname][i]["start"]:labels[fname][i]["end"]], rate=sr)

# Training neural networks

## Support functions

In [ ]:
# Randomise and split dataframe into X and Y
def feature_target_split(df, shuffle=True):
    if shuffle:
        df = df.sample(frac = 1)
    
    dataset = df.values
    X = dataset[:,1:].astype(float)
    
    Y = dataset[:,0]
    encoder = sklearn.preprocessing.LabelEncoder()
    encoder.fit(Y)
    Y = encoder.transform(Y)
    
    return X, Y

## Neural networks

In [ ]:
def two_layer_integrated(X):
    inputs = Input(shape= (X.shape[1],))
    layer = Dense(128, activation="relu")(inputs)
    outputs = Dense(1, activation="sigmoid")(layer)
    model = Model(inputs, outputs)
    model.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ["acc"])
    mc = ModelCheckpoint("best_model_whistle.hdf5", monitor="val_loss", verbose=1, save_best_only=True, mode="min")
    return model, mc

## Fitting network

In [ ]:
def train(X_train, X_test, Y_train, Y_test, version, feature_type, denoise, plot=False):
    
    # model training
    model, model_checkpoint = two_layer_integrated(X_train)
    history = model.fit(X_train, Y_train ,epochs=500, callbacks=[model_checkpoint], batch_size=32, 
                        validation_data=(X_test, Y_test))

    # load the best model weights
    model.load_weights('best_model_whistle.hdf5')
    
    # save model
    model.save("model_v" + str(version) + "_" + feature_type + "_" + str(denoise) + ".h5")

    # summarize history for loss
    if plot:
        plt.plot(history.history["acc"], c="#181818")
        plt.plot(history.history["val_acc"], c="#1ED760")
        plt.title("Model accuracy V" + str(version) + "_" + feature_type + "_" + str(denoise))
        plt.ylabel("Accuracy")
        plt.xlabel("Epoch")
        plt.legend(["Train", "Val"], loc="lower right")
        plt.ylim([0.83,1.01])
        plt.show()
        
    return model

def train_model(version, feature_type, denoise):
    # load data
    mass_data = build_features(version, feature_type, denoise)
    
    # train test split
    X, Y = feature_target_split(mass_data)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, 
                                                        random_state=12, shuffle=True)
    
    return X_test, Y_test, train(X_train, X_test, Y_train, Y_test, version, feature_type, denoise, plot=True)
        

X_test, Y_test, model = train_model(2, "mfcc", False)

## Confusion Matrix

In [ ]:
cm = confusion_matrix(Y_test, [1 if prediction > .5 else 0 for prediction in model.predict(X_test)[:,0]])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[False, True])
print("acc:", model.evaluate(X_test, Y_test, verbose=0)[1])
disp.plot(cmap="Greens")

## PCA Decomposition

In [ ]:
def pca_decompose(version, feature_type, denoise):
    # load data
    df = build_features(version, feature_type, denoise)
    
    # pca decomposition
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df.loc[:, df.columns != "target"])
    pca_df = pd.DataFrame(data = principalComponents, columns = ["dim_1", "dim_2"])
    pca_df.insert (0, "target", df["target"])
    
    # plot
    colors = ["#181818", "#1ED760"]
    for i, label in enumerate(pca_df["target"].unique()):
        pca_label = pca_df[pca_df["target"] == label]
        plt.scatter(pca_label["dim_1"], pca_label["dim_2"], label = bool(label), alpha=0.3, c = colors[i])
    plt.title("PCA decomposition V" + str(version) + "_" + feature_type + "_" + str(denoise))
    plt.xlim([-1,1.5])
    plt.ylim([-1,1.5])
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.show()
    
pca_decompose(2, "mfcc", False)

## K-fold validation

In [ ]:
# Kfold implementation, written by Paul Boersma
def kfold_index(df, k=5):
    N = len(df)
    minimum_number_of_points_per_slice = N // k
    remaining_number_of_points = N % k
    starting_point = 0
    out = []
    for islice in range(0, k):
        end_point = starting_point + minimum_number_of_points_per_slice + ( islice < remaining_number_of_points )
        out.append((starting_point, end_point))
        starting_point = end_point
    return out

# Run kfold for given featureset
def execute_kfold(version, feature_type, denoise, k=5):
    acc_train = []
    acc_test = []
    cm = np.zeros((2,2))
    df = build_features(version, feature_type, denoise)
    X, Y = feature_target_split(df)
    for start, end in kfold_index(df, k):
        X_train = np.concatenate((X[:start], X[end:]))
        Y_train = np.concatenate((Y[:start], Y[end:]))
        X_test = X[start:end]
        Y_test = Y[start:end]
        
        model = train(X_train, X_test, Y_train, Y_test, version, feature_type, denoise)
        acc_train.append(model.evaluate(X_train, Y_train, verbose=0)[1])
        acc_test.append(model.evaluate(X_test, Y_test, verbose=0)[1])
        cm += confusion_matrix(Y_test, [1 if prediction > .5 else 0 for prediction in model.predict(X_test)[:,0]])
        
    print('Accuracy on train data:', acc_train, np.mean(acc_train))
    print('Accuracy on test data:', acc_test, np.mean(acc_test))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[False, True])
    disp.plot(cmap="Greens", values_format='')

#execute_kfold(1, 'fft', False)

# Live detection

In [ ]:
# constants
FORMAT = pyaudio.paFloat32      # audio format (bytes per sample?)
CHANNELS = 1                    # single channel for microphone
SR = 48000                      # samples per second
CHUNK = int((SR / 1000) * STEP) # chunk size
WAVE_OUTPUT_FILENAME = "file.wav"

In [ ]:
def live_detect(feature_type, version, denoise, record_seconds=15, write=False):
    # load model
    try:
        model = load_model("model_v" + str(version) + "_" + feature_type + "_" + str(denoise) + ".h5")
    except:
        _, _, model = train_model(version, feature_type, denoise)
    
    # pyaudio class instance
    p = pyaudio.PyAudio()
    buffer = [False] * 10

    # stream object to get data from microphone
    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=SR,
        input=True,
        output=True,
        frames_per_buffer=CHUNK
    )

    print("recording...")
    frames = []

    for i in range(0, int(SR / CHUNK * record_seconds)):
        # read chunk
        data = stream.read(CHUNK)
        frames.append(data)
        
        # fetch features
        sample = np.frombuffer(data, dtype=np.float32)
        if feature_type == "fft":
            features = np.mean(np.abs(librosa.stft(sample, n_fft=512, hop_length=256, win_length=512)).T, axis=0)
        elif feature_type == "mfcc":
            features = np.mean(librosa.feature.mfcc(y=sample, sr=SR, n_mfcc=40).T, axis=0)
        
        # make prediction
        features = np.expand_dims(features, axis=0)
        is_whistle = model.predict(features, verbose=0)[0][0]
        
        # print current certainty
        buffer.pop(0)
        buffer.append(is_whistle > .5)
        clear_output(wait=True)
        print("whistle likelihood:", str((sum(buffer)/len(buffer)) * 100) + "%")
        
        
    print("finished recording")

    # stop recording
    stream.stop_stream()
    stream.close()
    p.terminate()

    # write to file
    if write:       
        waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(p.get_sample_size(FORMAT))
        waveFile.setframerate(SR)
        waveFile.writeframes(b''.join(frames))
        waveFile.close()
        
live_detect("mfcc", 2, False, write=False)